In [7]:
class BinaryTree:
    def __init__(self, key, leftTree=None, rightTree=None):
        self.key = key
        self.leftTree = leftTree
        self.rightTree = rightTree

    def setKey(self, key):
        self.key = key

    def getKey(self):
        return self.key

    def getLeftTree(self):
        return self.leftTree

    def getRightTree(self):
        return self.rightTree

    def insertLeft(self, key):
        if self.leftTree is None:
            self.leftTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, leftTree=self.leftTree)
            self.leftTree = new_node

    def insertRight(self, key):
        if self.rightTree is None:
            self.rightTree = BinaryTree(key)
        else:
            new_node = BinaryTree(key, rightTree=self.rightTree)
            self.rightTree = new_node

    def printPreorder(self, level=0):
        print(str(level * '-') + str(self.key))
        if self.leftTree is not None:
            self.leftTree.printPreorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPreorder(level + 1)
            
    def printInorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printInorder(level + 1)
        print(str(level * '-') + str(self.key))
        if self.rightTree is not None:
            self.rightTree.printInorder(level + 1)

    def printPostorder(self, level=0):
        if self.leftTree is not None:
            self.leftTree.printPostorder(level + 1)
        if self.rightTree is not None:
            self.rightTree.printPostorder(level + 1)
        print(str(level * '-') + str(self.key))


class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.isEmpty():
            return self.items.pop()
        else:
            raise IndexError("Pop from an empty stack")

    def peek(self):
        if not self.isEmpty():
            return self.items[-1]
        else:
            raise IndexError("Peek from an empty stack")

    def size(self):
        return len(self.items)


class Node:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, key, item):
        new_node = Node(key, item)
        if self.head is None:
            self.head = new_node
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = new_node

    def find(self, key):
        current = self.head
        while current:
            if current.key == key:
                return current.item
            current = current.next
        return None

    def sort(self):
        if self.head is None or self.head.next is None:
            return

        size = 1
        current = self.head
        while current.next:
            size += 1
            current = current.next

        step = 1
        while step < size:
            self.head = self.merge_pass(self.head, step)
            step *= 2

    def merge_pass(self, head, step):
        current = head
        prev_tail = None
        new_head = None

        while current:
            left = current
            right = self.split(left, step)
            current = self.split(right, step)

            merged = self.sorted_merge(left, right)

            if prev_tail:
                prev_tail.next = merged
            else:
                new_head = merged

            while prev_tail and prev_tail.next:
                prev_tail = prev_tail.next

        return new_head

    def split(self, head, step):
        if head is None:
            return None
        for _ in range(step - 1):
            if head.next is None:
                break
            head = head.next
        next_head = head.next
        head.next = None
        return next_head

    def sorted_merge(self, a, b):
        if a is None:
            return b
        if b is None:
            return a

        if a.key <= b.key:
            result = a
            result.next = self.sorted_merge(a.next, b)
        else:
            result = b
            result.next = self.sorted_merge(a, b.next)
        return result

In [8]:
import operator

def build_parse_tree(postfix_expr):
    token_list = postfix_expr.split()
    stack = Stack()
    
    for token in token_list:
        if token in "+-*/**":
            node = BinaryTree(token)
            node.rightTree = stack.pop()
            node.leftTree = stack.pop()
            stack.push(node)
        else:
            stack.push(BinaryTree(token))

    return stack.pop()

def evaluate(parse_tree):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv, '**': operator.pow}

    leftree = parse_tree.getLeftTree()
    rightree = parse_tree.getRightTree()

    if leftree and rightree:
        fn = ops[parse_tree.getKey()]
        return fn(evaluate(leftree), evaluate(rightree))
    else:
        return int(parse_tree.getKey())

In [9]:
def tokeniser(exp):
    raw = list(exp.replace(" ", ""))
    tokens = []
    i = 0
    
    while i < len(raw):
        if raw[i] == '*' and i + 1 < len(raw) and raw[i + 1] == '*':
            tokens.append('**')
            i += 2
        elif raw[i].isdigit():
            num = ''
            num += str(raw[i])
            i += 1
            while i < len(raw) and raw[i].isdigit():
                num += str(raw[i])
                i += 1
            tokens.append(num)
        else:
            tokens.append(raw[i])
            i += 1
    # debug: print(tokens)
    return tokens
    

def infix_to_postfix(infix_exp):
    prec = {}
    prec["**"] = 4
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1

    stack = Stack()
    postfix_list = []
    token_list = tokeniser(infix_exp)

    for token in token_list:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz" or token.isdigit():
            postfix_list.append(token)
        elif token == '(':
            stack.push(token)
        elif token == ')':
            top_token = stack.pop()
            while top_token != '(':
                postfix_list.append(top_token)
                top_token = stack.pop()
        else:
            while (not stack.isEmpty()) and (prec[stack.peek()] >= prec[token]):
                postfix_list.append(stack.pop())
            stack.push(token)

    while not stack.isEmpty():
        postfix_list.append(stack.pop())
    return " ".join(postfix_list)

def isoperator(value):
    return value in ['+', '-', '*', '/', '**']
def isvariable(char):
    return char.isalpha()

def tokenise(expression):
        exp = expression.replace(" ", "")
        print(exp)
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i+=2
            if exp[i] == '-':
                if (i != 0 and i + 1 < len(exp) and (((exp[i - 1]).isdigit() or isvariable((exp[i - 1])) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit()) or isvariable((exp[i + 1])) or isoperator(exp[i + 1]))):
                    tokens.append(exp[i])
                    i+=1
                else:
                    if (exp[i+1]).isdigit() or (exp[i+1]).isvariable():
                        tokens.append('(')
                        tokens.append(exp[i+1])
                        tokens.append('*')
                        tokens.append('-1')
                        tokens.append(')')
                        i+=2
                        
                    elif exp[i+1] == '(':
                        tokens.append('(')
                        tokens.append('-1')
                        tokens.append('*')
                        tokens.append(exp)
                        tokens.append(')')
                    else:
                        print("Error")
            
            else:
                tokens.append(exp[i])
                i+=1

        print(tokens)
        return tokens

In [10]:
variables = {}
def evaluate(parse_tree):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv, '**': operator.pow}

    leftree = parse_tree.getLeftTree()
    rightree = parse_tree.getRightTree()

    if leftree and rightree:
        fn = ops[parse_tree.getKey()]
        return fn(evaluate(leftree), evaluate(rightree))
    else:
        try:
            # Check if the node is a variable, and get its value from the dictionary
            if parse_tree.getKey() in variables:
                return evaluate_expression(variables[parse_tree.getKey()])
            else:
                # Otherwise, treat it as an integer
                return int(parse_tree.getKey())
        except ValueError:
            # Handle undefined variables or non-integer literals
            return None

def evaluate_expression(exp):
    postfix_exp = infix_to_postfix(exp)
    parse_tree = build_parse_tree(postfix_exp)
    return evaluate(parse_tree)

def add_or_modify_expression(var_name, exp):
    # Update the variables dictionary
    variables[var_name] = exp

def display_expressions():
    for var, exp in variables.items():
        try:
            result = evaluate_expression(exp)
            print(f"{var} = {exp} => {result}")
        except Exception as e:
            print(f"{var} = {exp} => None (Error: {e})")

# Example Usage
add_or_modify_expression('a', '5 + 6')
add_or_modify_expression('b', '2 - 5')
add_or_modify_expression('b', '((a) + 2)')  # This will update 'b'
add_or_modify_expression('c', 'x')      # 'x' is undefined, so it will throw an error

display_expressions()

5+6
['5', '+', '6']
a = 5 + 6 => 11
((a)+2)
['(', '(', 'a', ')', '+', '2', ')']
5+6
['5', '+', '6']
b = ((a) + 2) => 13
x
['x']
c = x => None


In [11]:
infix_expr = "20 90 +(24 ** 2 )"
postfix_expr = infix_to_postfix(infix_expr)
tree = build_parse_tree(postfix_expr)
result = evaluate(tree)
print("Result:", result)

2090+(24**2)
['2', '0', '9', '0', '+', '(', '2', '4', '**', '2', ')']
Result: 18


In [12]:
infix_expr = "3 + 4 * 2 ** 2"
postfix_expr = infix_to_postfix(infix_expr)
parse_tree = build_parse_tree(postfix_expr)

print("Preorder traversal of the parse tree:")
parse_tree.printPreorder()

3+4*2**2
['3', '+', '4', '*', '2', '**', '2']
Preorder traversal of the parse tree:
+
-3
-*
--4
--**
---2
---2
